In [1]:
import cv2
import numpy as np
import mediapipe as mp
import HandDetect as hd
import math

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
def nothing(q):
    pass

def is_camera_symbol(hand_landmarks1, hand_landmarks2):
    def get_landmark_pos(hand_landmarks, landmark_idx):
        return np.array([hand_landmarks.landmark[landmark_idx].x, hand_landmarks.landmark[landmark_idx].y])

    # Define landmarks to form the rectangle (index finger tips and thumbs)
    index_tip1 = get_landmark_pos(hand_landmarks1, mp_hands.HandLandmark.INDEX_FINGER_TIP)
    thumb_tip1 = get_landmark_pos(hand_landmarks1, mp_hands.HandLandmark.THUMB_TIP)
    index_tip2 = get_landmark_pos(hand_landmarks2, mp_hands.HandLandmark.INDEX_FINGER_TIP)
    thumb_tip2 = get_landmark_pos(hand_landmarks2, mp_hands.HandLandmark.THUMB_TIP)

    # Calculate distances between landmarks
    distance_it1 = np.linalg.norm(index_tip1 - thumb_tip2)
    distance_it2 = np.linalg.norm(thumb_tip1 - index_tip2)

    # Check if the distances are within a certain range (indicating a rectangle formation)
    if distance_it1 < 0.1 and distance_it2 < 0.1:
        return True

    return False

def snapshot(img):
    rgb_frame = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # Draw the hand annotations on the image
    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) == 2:
        hand_landmarks1 = results.multi_hand_landmarks[0]
        hand_landmarks2 = results.multi_hand_landmarks[1]
        # Check for the "camera symbol" gesture
        return is_camera_symbol(hand_landmarks1, hand_landmarks2)

def create_canvas():
    temp = np.zeros((720,1100,3),dtype='uint8')                                                   # see that size of webcam is 1100*720 so we initialise paintW with rows = 720
    paintW = np.zeros((720,1100,3),dtype='uint8')+255
    paintW = cv2.rectangle(paintW, (40,1),  (180,90), (0,0,0), 2)
    paintW = cv2.rectangle(paintW, (210,1), (350,90), (255,0,0), 2)
    paintW = cv2.rectangle(paintW, (380,1), (520,90), (0,255,0), 2)
    paintW = cv2.rectangle(paintW, (550,1), (690,90), (0,0,255), 2)
    paintW = cv2.rectangle(paintW, (720,1), (860,90), (255,0,255), 2)
    paintW = cv2.rectangle(paintW, (890,1), (1030,90),(0,255,255), 2)
    cv2.putText(paintW, "CLEAR",  (85, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(paintW, "BLUE",  (260, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(paintW, "GREEN", (425, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(paintW, "RED",   (605, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(paintW, "PINK",  (770, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(paintW, "YELLOW",(930, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    return temp,paintW

cap = cv2.VideoCapture(0)                                                                         # this always returns array with order col*rows and not rows*col
det = hd.handDetector(det_conf = 0.45)
w = 1100                                                                                          # represents col
h=720                                                                                             # represents rows
snap_ct=0
draw_color = [(255,0,0),  (0,255,0),  (0,0,255),  (255,0,255), (0,255,255)]
color_ind = 0

temp, paintW = create_canvas()
cv2.namedWindow('Canvas')
cv2.createTrackbar('Brush Size', 'Canvas', 5, 20, nothing)

xp, yp = 0,0
while True:
    _,img = cap.read()
    img = cv2.flip(img,1)
    img = det.findHands(img, False,w, h)
    lmlist = det.findPos(0,w,h)
    fing_up = det.fingUp()
    print(fing_up)
    if (len(lmlist)!=0):
        x1,y1 = lmlist[8][1],lmlist[8][2]       # coordinates of index finger
        x2,y2 = lmlist[12][1], lmlist[12][2]    # coordinates of middle finger

        if (fing_up[0]==1 and fing_up[1]==1):
            xp,yp = x1,y1
            if y1<90:
                if 40 < x1 < 180:
                    temp, paintW = create_canvas()
                elif 210 < x1 < 350:
                    color_ind = 0
                elif 380 < x1 < 520:
                    color_ind = 1
                elif 550 < x1 < 690:
                    color_ind = 2
                elif 720 < x1 < 860:
                    color_ind = 3
                elif 890 < x1 < 1030:
                    color_ind = 4

        elif (fing_up[0]==1 and fing_up[1]!=1):
            brush_thickness = cv2.getTrackbarPos('Brush Size', 'Canvas')
            cv2.circle(img, (x1, y1), 10, draw_color[color_ind], 10)                # pass img, string to pe printed, location, font type, font size, font color & font thickn.
            if xp==0 and yp==0:
                xp,yp = x1,y1
            cv2.line(img,   (xp,yp),(x1,y1),draw_color[color_ind],brush_thickness)
            cv2.line(paintW,(xp,yp),(x1,y1),draw_color[color_ind],brush_thickness)
            cv2.line(temp,  (xp,yp),(x1,y1),draw_color[color_ind],brush_thickness)
            xp,yp = x1,y1

    img_gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
    _,img_inv = cv2.threshold(img_gray,25,255,cv2.THRESH_BINARY_INV)
    img_inv = cv2.cvtColor(img_inv,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img,img_inv)
    img = cv2.bitwise_or(img,temp)

    img = cv2.rectangle(img, (40,1),  (180,90), (0,0,0), 2)
    img = cv2.rectangle(img, (210,1), (350,90), (255,0,0), 2)
    img = cv2.rectangle(img, (380,1), (520,90), (0,255,0), 2)
    img = cv2.rectangle(img, (550,1), (690,90), (0,0,255), 2)
    img = cv2.rectangle(img, (720,1), (860,90), (255,0,255), 2)
    img = cv2.rectangle(img, (890,1), (1030,90),(0,255,255), 2)
    cv2.putText(img, "CLEAR",  (85, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "BLUE",  (260, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "GREEN", (425, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "RED",   (605, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "PINK",  (770, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "YELLOW",(930, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

    if (snapshot(img)):
        snapshot_filename = f'snapshot_{snap_ct}.png'
        cv2.imwrite(snapshot_filename, paintW)
        print(f'Snapshot saved as {snapshot_filename}')
        snap_ct+=1
        
    cv2.imshow("Webcam",img)
    cv2.imshow("Canvas",paintW)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()